In [1]:
import pandas as pd
import json

def merge(filename: str, station_name: pd.DataFrame) -> pd.DataFrame:
    # 讀取CSV文件
    df1 = pd.read_csv(filename)
    # 重命名欄位以便更容易理解
    df2 = df1.rename(columns={'trnOpDate':'日期',
                              'staCode':'車站代碼',
                              'gateInComingCnt':'進站人數',
                              'gateOutGoingCnt':'出站人數'})
    # 將日期欄位轉換為日期格式
    df2['日期'] = pd.to_datetime(df2['日期'].astype(str))
    # 合併數據框 df2 和 station_name，根據車站代碼進行匹配
    result_df = pd.merge(df2, station_name, left_on='車站代碼', right_on='編碼', how='left')
    return result_df

# 打開並讀取車站基本資料的 JSON 文件
with open('每日各站進出站人數/車站基本資料集.json', encoding='utf-8') as file:
    json_data: list[dict] = json.load(file)

# 將 JSON 數據轉換為 Pandas 數據框，並選擇特定的欄位
stations_name = pd.DataFrame(json_data, columns=['stationCode', 'stationName', 'stationAddrTw'])
# 重命名欄位以便更容易理解
stations_name1 = stations_name.rename(columns={'stationCode':'編碼',
                                               'stationName':'車站名稱',
                                               'stationAddrTw':'地址'})
# 將車站編碼轉換為整數格式
stations_name1['編碼'] = stations_name1['編碼'].astype(int)


In [3]:
import os
# 取得目前執行檔案的絕對路徑
current_path = os.path.abspath(__name__)
# 取得目前執行的目錄的絕對路徑
main_path = os.path.dirname(current_path)

# 資料夾名稱
'每日各站進出站人數'
# 取得放資料的目錄
dataFolder_path = os.path.join(main_path, '每日各站進出站人數')
# 取得資料目錄的所有檔案名稱
filenames = os.listdir(dataFolder_path)
# 過濾出包含 '每日各站進出站人數' 字串的檔案名稱
names = list(filter(lambda name: '每日各站進出站人數' in name, filenames))

# 使用列表生成式取得每個檔案的完整路徑
abs_names = [os.path.join(dataFolder_path, name) for name in names]
# 使用列表生成式讀取每個檔案並合併數據
all_data = [merge(filename=abs_n, station_name=stations_name1) for abs_n in abs_names]

# 將所有合併後的數據框連接成一個數據框
result = pd.concat(all_data, ignore_index=True)
# 設定日期為索引
result1 = result.set_index('日期')
# 根據日期排序並重新排列欄位順序
result2 = result1.sort_index().reindex(columns=['車站名稱', '地址', '進站人數', '出站人數'])
result2.to_csv("2019~2023進出站人數.csv")